<a href="https://colab.research.google.com/github/SABRIS13/Machine_Learning_Production/blob/main/Pyspark_proyecto_Stanford_open_police_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Standford open policing project

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=1387c70f9afcb0cbfbd5ae95903e706572eb39e51b27be3a0a9b0d920c8e39cd
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
# si estamos trabajando en colab, 
# debemos correr las siguientes líneas:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# si estamos trabajando en colab, 
# podemos montar el drive
# from google.colab import drive
# drive.mount('/content/drive')

## Antecedentes

Trabajaremos datos del proyecto abierto _stanford open policing project_, que cuenta con data de 31 estados de los EUA, pero específicamente trabajaremos con datos del estado de Rhod Island.

Este proyecto está relacionado con una serie de investigaciones y publicaciones muy famosas todas hechas en R, pero en este proyecto trabajaremos en spark, si quieren conocer más, pueden visitar la [página oficial](https://openpolicing.stanford.edu/) del proyecto

## Preparación de los datos

Examina y limpia los datos:

In [3]:
from pyspark.sql import functions as F

In [4]:
!wget -O police_activity.csv https://raw.githubusercontent.com/edroga/Datasets_for_projects/main/police_activity.csv

--2023-03-03 01:06:26--  https://raw.githubusercontent.com/edroga/Datasets_for_projects/main/police_activity.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9128997 (8.7M) [text/plain]
Saving to: ‘police_activity.csv’

police_activity.csv 100%[===================>]   8.71M  --.-KB/s    in 0.07s   

2023-03-03 01:06:26 (124 MB/s) - ‘police_activity.csv’ saved [9128997/9128997]



In [87]:
# Carga los datos
ri = spark.read.csv('police_activity.csv',
                    sep=',', 
                    header=True, 
                    inferSchema=True)

In [6]:
# Examina la estructura del DataFrame
ri.show(2)

+-----+-------------------+-------------------+-----------+-------------+-----------+--------------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|state|          stop_date|          stop_time|county_name|driver_gender|driver_race|       violation_raw|violation|search_conducted|search_type|stop_outcome|is_arrested|stop_duration|drugs_related_stop|district|
+-----+-------------------+-------------------+-----------+-------------+-----------+--------------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|   RI|2005-01-04 00:00:00|2023-03-03 12:55:00|       null|            M|      White|Equipment/Inspect...|Equipment|           false|       null|    Citation|      false|     0-15 Min|             false| Zone X4|
|   RI|2005-01-23 00:00:00|2023-03-03 23:15:00|       null|            M|      White|            Speeding| Speeding|           false|       null|   

In [7]:
# Examina el tipo de datos del DataFrame
ri.dtypes

[('state', 'string'),
 ('stop_date', 'timestamp'),
 ('stop_time', 'timestamp'),
 ('county_name', 'string'),
 ('driver_gender', 'string'),
 ('driver_race', 'string'),
 ('violation_raw', 'string'),
 ('violation', 'string'),
 ('search_conducted', 'boolean'),
 ('search_type', 'string'),
 ('stop_outcome', 'string'),
 ('is_arrested', 'boolean'),
 ('stop_duration', 'string'),
 ('drugs_related_stop', 'boolean'),
 ('district', 'string')]

In [10]:
ri.printSchema()  #útil printSchema cuando tenemos datos anidados

root
 |-- state: string (nullable = true)
 |-- stop_date: timestamp (nullable = true)
 |-- stop_time: timestamp (nullable = true)
 |-- county_name: string (nullable = true)
 |-- driver_gender: string (nullable = true)
 |-- driver_race: string (nullable = true)
 |-- violation_raw: string (nullable = true)
 |-- violation: string (nullable = true)
 |-- search_conducted: boolean (nullable = true)
 |-- search_type: string (nullable = true)
 |-- stop_outcome: string (nullable = true)
 |-- is_arrested: boolean (nullable = true)
 |-- stop_duration: string (nullable = true)
 |-- drugs_related_stop: boolean (nullable = true)
 |-- district: string (nullable = true)



In [12]:
#Conocer las columnas
ri.columns

['state',
 'stop_date',
 'stop_time',
 'county_name',
 'driver_gender',
 'driver_race',
 'violation_raw',
 'violation',
 'search_conducted',
 'search_type',
 'stop_outcome',
 'is_arrested',
 'stop_duration',
 'drugs_related_stop',
 'district']

In [13]:
#Conteo de registros
ri.count()

91741

In [14]:
# Examine las dimensiones del DataFrame (Métodos similares a shape en pandas)
ri.count(), len(ri.columns)

(91741, 15)

In [15]:
# Usa el método describe sobre el DataFrame
ri.describe().show()

+-------+-----+-----------+-------------+-----------+-------------+---------+--------------------+-------------+-------------+--------+
|summary|state|county_name|driver_gender|driver_race|violation_raw|violation|         search_type| stop_outcome|stop_duration|district|
+-------+-----+-----------+-------------+-----------+-------------+---------+--------------------+-------------+-------------+--------+
|  count|91741|          0|        86536|      86539|        86539|    86539|                3307|        86539|        86539|   91741|
|   mean| null|       null|         null|       null|         null|     null|                null|         null|         null|    null|
| stddev| null|       null|         null|       null|         null|     null|                null|         null|         null|    null|
|    min|   RI|       null|            F|      Asian|          APB|Equipment|  Incident to Arrest|Arrest Driver|     0-15 Min| Zone K1|
|    max|   RI|       null|            M|      W

In [25]:
#Seleccionar columnas
ri.select('driver_race','driver_gender').show(5)

+-----------+-------------+
|driver_race|driver_gender|
+-----------+-------------+
|      White|            M|
|      White|            M|
|      White|            M|
|      White|            M|
|      White|            F|
+-----------+-------------+
only showing top 5 rows



In [16]:
# Cuenta el número de missings por cada columna en el DataFrame
(ri
 .select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in ri.columns])
 .show())

+-----+---------+---------+-----------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|state|stop_date|stop_time|county_name|driver_gender|driver_race|violation_raw|violation|search_conducted|search_type|stop_outcome|is_arrested|stop_duration|drugs_related_stop|district|
+-----+---------+---------+-----------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|    0|        0|        0|      91741|         5205|       5202|         5202|     5202|               0|      88434|        5202|       5202|         5202|                 0|       0|
+-----+---------+---------+-----------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+



In [23]:
# Elimina las columnas 'county_name', 'state'
ri = ri.drop(*['county_name', 'state'])

In [26]:
# Nuevamente examine las dimensiones del DataFrame
ri.count(), len(ri.columns)

(91741, 13)

In [27]:
# Elimina los registros que tengan missings en la columna 'driver_gender'
ri = ri.dropna(subset=['driver_gender'])

In [28]:
# Nuevament cuenta el número de missings por cada columna en el DataFrame
(ri
 .select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in ri.columns])
 .show())

+---------+---------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|stop_date|stop_time|driver_gender|driver_race|violation_raw|violation|search_conducted|search_type|stop_outcome|is_arrested|stop_duration|drugs_related_stop|district|
+---------+---------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|        0|        0|            0|          0|            0|        0|               0|      83229|           0|          0|            0|                 0|       0|
+---------+---------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+



In [29]:
# Nuevamente examine las dimensiones del DataFrame
ri.count(), len(ri.columns)

(86536, 13)

## Tipos de datos correctos

In [30]:
# Examina las primeras 4 filas del DataFrame
ri.show(4)

+-------------------+-------------------+-------------+-----------+--------------------+---------+----------------+-----------+-------------+-----------+-------------+------------------+--------+
|          stop_date|          stop_time|driver_gender|driver_race|       violation_raw|violation|search_conducted|search_type| stop_outcome|is_arrested|stop_duration|drugs_related_stop|district|
+-------------------+-------------------+-------------+-----------+--------------------+---------+----------------+-----------+-------------+-----------+-------------+------------------+--------+
|2005-01-04 00:00:00|2023-03-03 12:55:00|            M|      White|Equipment/Inspect...|Equipment|           false|       null|     Citation|      false|     0-15 Min|             false| Zone X4|
|2005-01-23 00:00:00|2023-03-03 23:15:00|            M|      White|            Speeding| Speeding|           false|       null|     Citation|      false|     0-15 Min|             false| Zone K3|
|2005-02-17 00:00:00

In [31]:
# Examina los data types del DataFrame
ri.dtypes

[('stop_date', 'timestamp'),
 ('stop_time', 'timestamp'),
 ('driver_gender', 'string'),
 ('driver_race', 'string'),
 ('violation_raw', 'string'),
 ('violation', 'string'),
 ('search_conducted', 'boolean'),
 ('search_type', 'string'),
 ('stop_outcome', 'string'),
 ('is_arrested', 'boolean'),
 ('stop_duration', 'string'),
 ('drugs_related_stop', 'boolean'),
 ('district', 'string')]

In [32]:
# transforma 'stop_date' and 'stop_time' en una nueva columna con formato datetime llamada stop_datetime
ri = (ri.withColumn('stop_datetime', F.concat_ws(' ', 'stop_date', 'stop_time'))
        .withColumn('stop_datetime', F.from_unixtime(F.unix_timestamp(F.col('stop_datetime'), 'yyyy-MM-dd HH:mm'))))

In [36]:
ri.show()

+-----+-------------------+-------------------+-----------+-------------+-----------+--------------------+----------------+----------------+-----------+-------------+-----------+-------------+------------------+--------+
|state|          stop_date|          stop_time|county_name|driver_gender|driver_race|       violation_raw|       violation|search_conducted|search_type| stop_outcome|is_arrested|stop_duration|drugs_related_stop|district|
+-----+-------------------+-------------------+-----------+-------------+-----------+--------------------+----------------+----------------+-----------+-------------+-----------+-------------+------------------+--------+
|   RI|2005-01-04 00:00:00|2023-03-03 12:55:00|       null|            M|      White|Equipment/Inspect...|       Equipment|           false|       null|     Citation|      false|     0-15 Min|             false| Zone X4|
|   RI|2005-01-23 00:00:00|2023-03-03 23:15:00|       null|            M|      White|            Speeding|        Sp

## ¿Hay evidencia de diferentes delitos cometidos por género?

Crea una tabla con el siguiente formato:


|violation| F| M| Total| Fp100_of_Total| Mp100_of_Total|
| --| --| --| --| --| --|
| Equipment| 2501| 8420| 10921| 0.23| 0.77|
| Other| 707| 3702| 4409| 0.16| 0.84|
| ...| ...| ...| ...| ...| ...|

En donde:

+ __Violation__ son las violaciones cometidas
+ __F__ cantidad de mujeres que cometen esa violación
+ __M__ cantidad de hombres que cometen esa violación
+ __Total__ suma de __F__ + __M__
+ __Fp100_of_Total__ división de __F__ / __Total__
+ __Mp100_of_Total__ división de __F__ y __Total__


In [55]:
t1 = (ri.dropna(subset=['violation']).groupBy(['violation', 'driver_gender'])
 .count()
 .groupBy("violation")
 .pivot("driver_gender")
 .sum("count")
 .withColumn('Total', F.col('F') + F.col('M'))  #Crear columna nueva: "nombre" y "Operación de la columna"
 .withColumn('Fp100_of_Total', F.round(F.col('F')/F.col('Total'), 2))
 .withColumn('Mp100_of_Total', F.round(F.col('M')/F.col('Total'), 2)))

t1.show()

+-------------------+----+-----+-----+-----+--------------+--------------+
|          violation|null|    F|    M|Total|Fp100_of_Total|Mp100_of_Total|
+-------------------+----+-----+-----+-----+--------------+--------------+
|          Equipment|   1| 2501| 8420|10921|          0.23|          0.77|
|              Other|   1|  707| 3702| 4409|          0.16|          0.84|
|   Moving violation|null| 3286|12938|16224|           0.2|           0.8|
|Registration/plates|null| 1056| 2647| 3703|          0.29|          0.71|
|           Speeding|   1|15646|32777|48423|          0.32|          0.68|
|          Seat belt|null|  578| 2278| 2856|           0.2|           0.8|
+-------------------+----+-----+-----+-----+--------------+--------------+



Agrega dos columnas más a la tabla anterior y llámalas:

* __Fp100__ es la división de __F__ / __SUMA(columna F)__
* __Mp100__ es la división de __M__ / __SUMA(columna M)__

In [56]:
TF = t1.select((F.sum('F'))).first()[0]
TM = t1.select((F.sum('M'))).first()[0]

In [58]:
t1.withColumn('Fp100', F.round(F.col('F')/TF, 2)).withColumn('Mp100', F.round(F.col('M')/TM, 2)).show()

+-------------------+----+-----+-----+-----+--------------+--------------+-----+-----+
|          violation|null|    F|    M|Total|Fp100_of_Total|Mp100_of_Total|Fp100|Mp100|
+-------------------+----+-----+-----+-----+--------------+--------------+-----+-----+
|          Equipment|   1| 2501| 8420|10921|          0.23|          0.77| 0.11| 0.13|
|              Other|   1|  707| 3702| 4409|          0.16|          0.84| 0.03| 0.06|
|   Moving violation|null| 3286|12938|16224|           0.2|           0.8| 0.14| 0.21|
|Registration/plates|null| 1056| 2647| 3703|          0.29|          0.71| 0.04| 0.04|
|           Speeding|   1|15646|32777|48423|          0.32|          0.68| 0.66| 0.52|
|          Seat belt|null|  578| 2278| 2856|           0.2|           0.8| 0.02| 0.04|
+-------------------+----+-----+-----+-----+--------------+--------------+-----+-----+



## ¿Impacta el género en quiénes reciben más multas?

## Analiza el género cuando las violaciones son por _speeding_

In [62]:
# Filtra el dataset por violation == 'speedong'
gender_and_speeding = ri.filter(F.col('violation')=='Speeding')  #Tambien se puede poner where en vez de filter
gender_and_speeding.show(5)

+-----+-------------------+-------------------+-----------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|state|          stop_date|          stop_time|county_name|driver_gender|driver_race|violation_raw|violation|search_conducted|search_type|stop_outcome|is_arrested|stop_duration|drugs_related_stop|district|
+-----+-------------------+-------------------+-----------+-------------+-----------+-------------+---------+----------------+-----------+------------+-----------+-------------+------------------+--------+
|   RI|2005-01-23 00:00:00|2023-03-03 23:15:00|       null|            M|      White|     Speeding| Speeding|           false|       null|    Citation|      false|     0-15 Min|             false| Zone K3|
|   RI|2005-02-17 00:00:00|2023-03-03 04:15:00|       null|            M|      White|     Speeding| Speeding|           false|       null|    Citation|      false|     0-15 Min

In [65]:
# A partir del DataFrame anterior, calcula en qué terminó el encuentro con la policia 'stop_outcome' por género
Outcomes_by_gender =( gender_and_speeding.groupBy(['driver_gender', 'stop_outcome'])
                      .count()
                      .groupBy('stop_outcome')
                      .pivot('driver_gender')
                      .sum('count'))
Outcomes_by_gender.show(5)

+----------------+----+-----+-----+
|    stop_outcome|null|    F|    M|
+----------------+----+-----+-----+
|        Citation|   1|14898|30961|
|Arrest Passenger|null|   10|   42|
|             N/D|null|   15|   32|
|       No Action|null|    6|   35|
|   Arrest Driver|null|   90|  521|
+----------------+----+-----+-----+
only showing top 5 rows



A partir del DataFrame anterior, calcula las siguientes variables:

* __Fp100__ = __F__ / __SUMA(columna F)__
* __Mp100__ = __F__ / __SUMA(columna M)__

In [66]:
OTF = Outcomes_by_gender.select((F.sum('F'))).first()[0]
OTM = Outcomes_by_gender.select((F.sum('M'))).first()[0]

In [70]:
Outcomes_by_gender.withColumn('Fp100', F.round(F.col('F')/OTF, 2)).withColumn('Mp100', F.round(F.col('M')/OTM, 2)).show()

+----------------+----+-----+-----+-----+-----+
|    stop_outcome|null|    F|    M|Fp100|Mp100|
+----------------+----+-----+-----+-----+-----+
|        Citation|   1|14898|30961| 0.95| 0.94|
|Arrest Passenger|null|   10|   42|  0.0|  0.0|
|             N/D|null|   15|   32|  0.0|  0.0|
|       No Action|null|    6|   35|  0.0|  0.0|
|   Arrest Driver|null|   90|  521| 0.01| 0.02|
|         Warning|null|  627| 1186| 0.04| 0.04|
+----------------+----+-----+-----+-----+-----+



## Calcula el search-rate

Obtén la siguiente tabla:

|driver_gender|false|true|Rate|
|--|--|--|--|
|F|23318| 456|0.019180617481282074|
|M|59911|2851| 0.04542557598546892|

donde:

* __false__ es la cantidad de veces que no se realizó una 'search_conducted'
* __true__ es la cantidad de veces que se llevó a cabo una 'search_conducted'
* __rate__ es __true__ / (__true__ + __false__)

In [71]:
(ri.groupBy(['search_conducted', 'driver_gender'])
 .count()
 .groupBy("driver_gender")
 .pivot("search_conducted")
 .sum("count")
 .withColumn('Rate', F.col('true')/(F.col('true')+ F.col('false')))
 ).show()

+-------------+-----+----+--------------------+
|driver_gender|false|true|                Rate|
+-------------+-----+----+--------------------+
|            F|23318| 456|0.019180617481282074|
|         null| 5205|null|                null|
|            M|59911|2851| 0.04542557598546892|
+-------------+-----+----+--------------------+



## Análisis de violation

Incorpora al DataFrame anterior la variable violation y comprueba si el search-rate varía por clase y género, y responde si los hombres y las mujeres tienden a cometer distintos delitos.

In [72]:
from pyspark.sql.types import IntegerType

In [73]:
(ri.withColumn('search_conducted', F.col('search_conducted').cast(IntegerType()))
 .groupBy(['violation', 'driver_gender'])
 .agg(F.avg('search_conducted').alias('mean'))
 .withColumn('mean', F.round(F.col('mean'),3 ) )
 .groupBy("violation")
 .pivot("driver_gender")
 .sum("mean")
#  .withColumn('Rate', F.col('true')/(F.col('true')+ F.col('false')))
 ).show()

+-------------------+----+-----+-----+
|          violation|null|    F|    M|
+-------------------+----+-----+-----+
|               null| 0.0| null| null|
|          Equipment| 0.0| 0.04|0.071|
|              Other| 0.0|0.041|0.046|
|   Moving violation|null|0.039|0.062|
|Registration/plates|null|0.055|0.109|
|           Speeding| 0.0|0.008|0.028|
|          Seat belt|null|0.017|0.035|
+-------------------+----+-----+-----+



## Análisis de los frisks

Analiza la frecuencia de las clases en search-rate:

In [75]:
(ri
 .groupBy(['search_type'])
 .count()
 ).show(truncate=False)

+--------------------------------------------------------+-----+
|search_type                                             |count|
+--------------------------------------------------------+-----+
|Protective Frisk                                        |164  |
|Inventory,Probable Cause,Protective Frisk               |1    |
|Probable Cause                                          |924  |
|Inventory,Probable Cause,Reasonable Suspicion           |1    |
|Incident to Arrest,Protective Frisk                     |33   |
|Inventory,Protective Frisk                              |12   |
|null                                                    |88434|
|Incident to Arrest,Probable Cause,Reasonable Suspicion  |5    |
|Incident to Arrest,Protective Frisk,Reasonable Suspicion|2    |
|Incident to Arrest,Inventory,Probable Cause             |35   |
|Probable Cause,Protective Frisk                         |35   |
|Incident to Arrest                                      |1290 |
|Probable Cause,Protectiv

### Compara frisk rates por género

1. Elimina los nulos de la variable search_type
2. Detecta dónde hay registros relacionados con 'frisk'
3. Agrupa por género
4. Obtén el promedio por género de los 'frisk'

In [76]:
(ri
 .dropna(subset = ['search_type'])
 .withColumn('frisk', F.regexp_extract(F.lower(F.col('search_type') ), '(frisk)', 1))
 .withColumn('frisk', F.when(F.col('frisk')=='frisk', 1)
                       .otherwise(0))

 .groupBy(['driver_gender'])
 .agg(F.avg('frisk'))
 ).show(100, False)

+-------------+-------------------+
|driver_gender|avg(frisk)         |
+-------------+-------------------+
|F            |0.07456140350877193|
|M            |0.0943528586460891 |
+-------------+-------------------+



### Calcula el promedio de las horas de arresto por día de la semana

In [94]:
hr_arrest = (ri
             .withColumn('is_arrested', F.when(F.col('is_arrested')==True, 1)
                                      .otherwise(0))
             .withColumn('weekday', F.date_format(F.col('stop_date'), 'E'))
             .groupBy(['weekday', F.hour('stop_time').alias('hr')])
             .agg(F.avg('is_arrested').alias('avg'))
             .groupBy('hr')
             .pivot('weekday')
             .sum('avg')
             .toPandas()
             .sort_values(by='hr', ascending = False)
             )
hr_arrest

,hr,Fri,Mon,Sat,Sun,Thu,Tue,Wed
15,23,0.063953,0.035568,0.047500,0.046481,0.039897,0.040691,0.035714
1,22,0.069021,0.037773,0.055268,0.047191,0.043478,0.038153,0.020202
18,21,0.059441,0.036364,0.053232,0.072289,0.071730,0.057471,0.054878
7,20,0.069159,0.028200,0.028668,0.028640,0.044211,0.023641,0.023364
9,19,0.039720,0.024590,0.021505,0.044218,0.039216,0.015823,0.031700
23,18,0.043165,0.034759,0.020690,0.048148,0.040956,0.048571,0.019499
11,17,0.059432,0.036446,0.050847,0.050139,0.019337,0.011933,0.037406
4,16,0.023913,0.028933,0.036545,0.044014,0.026316,0.032086,0.038760
10,15,0.025039,0.025199,0.031888,0.027548,0.032885,0.032742,0.028090
20,14,0.025572,0.031690,0.034606,0.033215,0.022107,0.034653,0.019737


In [95]:
hr_arrest['hrstr'] = hr_arrest.hr.apply(lambda x: 'hr_' + str(x))
hr_arrest['hr_dummy'] = hr_arrest.hr.apply(lambda x: x - 7 if x - 7 >= 0 else x + 17)
hr_arrest.sort_values(by='hr_dummy', ascending=False, inplace=True)
hr_arrest

,hr,Fri,Mon,Sat,Sun,Thu,Tue,Wed,hrstr,hr_dummy
5,6,0.010830,0.022088,0.011765,0.006826,0.020121,0.009506,0.009960,hr_6,23
8,5,0.000000,0.000000,0.150000,0.083333,0.045455,0.000000,0.047619,hr_5,22
13,4,0.029412,0.041667,0.083333,0.034483,0.081633,0.000000,0.000000,hr_4,21
6,3,0.047619,0.047244,0.066079,0.065574,0.050000,0.042857,0.049645,hr_3,20
21,2,0.043478,0.051282,0.065180,0.055755,0.057471,0.041667,0.068249,hr_2,19
2,1,0.061728,0.067474,0.072254,0.056738,0.064309,0.044750,0.046312,hr_1,18
22,0,0.054475,0.036775,0.045103,0.055276,0.037940,0.042135,0.060907,hr_0,17
15,23,0.063953,0.035568,0.047500,0.046481,0.039897,0.040691,0.035714,hr_23,16
1,22,0.069021,0.037773,0.055268,0.047191,0.043478,0.038153,0.020202,hr_22,15
18,21,0.059441,0.036364,0.053232,0.072289,0.071730,0.057471,0.054878,hr_21,14


In [ ]:
# Grafica la tabla anterior como un radar chart

In [97]:
import plotly.graph_objects as go

categories = hr_arrest.hrstr

Fri = list(hr_arrest.Fri)
Sat = list(hr_arrest.Sat)
Sun = list(hr_arrest.Sun)
Mon = list(hr_arrest.Mon)
Tue = list(hr_arrest.Tue)
Wed = list(hr_arrest.Wed)
Thu = list(hr_arrest.Thu)

f = go.Figure(
    data=[
        go.Scatterpolar(r=Fri, theta=categories, name='Fri'),
        go.Scatterpolar(r=Sat, theta=categories, name='Sat'),
        go.Scatterpolar(r=Sun, theta=categories, name='Sun'),
        go.Scatterpolar(r=Mon, theta=categories, name='Mon'),
        go.Scatterpolar(r=Tue, theta=categories, name='Tue'),
        go.Scatterpolar(r=Wed, theta=categories, name='Wed'),
        go.Scatterpolar(r=Thu, theta=categories, name='Thu')
    ],
    layout=go.Layout(
        title=go.layout.Title(text='Weekday comparison'),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
)

f.show()